# Capstone Project - The Battle of Neighborhoods

### Table of contents
___
1. *Problem Statement*
2. *Data description*
3. *Methodology*
4. *Results*
5. *Discussion and Coclusions*

### 1. Problem Statement 
___
I work for a global mulinational located in Milan, IT but due to COVID I left my apartment. Probably, with the vaccination campaigns in place, we will return to the office. Therefore, I'm looking for a new apartment in Milan and I would like to use this opportunity to practice my learnings in Coursera, particularly with Foursquare, in order to answer the relevant questions arisen.
The key question that I would address whitin this project is the following: How can I find a convenient and enjoyable place that fit my interests?
In order to make a comparison and an evaluation of the rental option, below is a list of some "constraints" based on what I am looking for:
- Apartment must be one or two room flat
- Desired location is near a metro station within 1km radius
- Price of rent not exceed €1,200 per month
- Nice to have venues such as gym, food shops and restaurants
Finding an apartment in Milan is always an hard job, especially for one and two rooms flats. Therefore, I believe that this work could be useful first for helping me to find a solution and, in general, for anyone moving to other large city in Italy. 

### 2. Data description 
___
To empirically investigate the research question identified in this study, the following data is required:
- List of Boroughs and Neighborhoods of Milan with their geodata (latitude and longitude)
- List of Subway metro stations in Milan with their address location
- List of apartments for rent in Milan including their price
- List of venues for each Milan neighborhood

To retrieve the list of boroughs and neighborhoods of Milan, the Wikipedia page (URL: https://en.wikipedia.org/wiki/Municipalities_of_Milan) will be used, scraping through the python library BeautifulSoup. To get the list of Subway metro stations in Milan, the CKAN Data API will be used to query the open data provided by the Government. A detailed view of the dataset used in this study can be found at the following [link](https://dati.comune.milano.it/dataset/ds535_atm-fermate-linee-metropolitane). To fetch apartments for rent, including their price, a real estate API will be used. Finally, venues will be collected via the Foursquare API.

#### 2.1 List of Boroughs and Neighborhoods of Milan with their geodata (latitude and longitude)

In [77]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

url= "https://en.wikipedia.org/wiki/Municipalities_of_Milan"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table_contents = []
table = soup.find("table",{"class":"wikitable sortable"})
table_body = table.find('tbody')

rows = table_body.find_all('tr')

for row in rows:
    cell = {}
    cols = row.findAll('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
        cell['Borough'] = cols[0]
        cell['BoroughName'] = cols[1]
        cell['Neighborhood'] = cols[5]
        table_contents.append(cell)

milan_neighborhood=pd.DataFrame(table_contents)


milan_neighborhood.head()

,Borough,BoroughName,Neighborhood
0,1,Centro storico,"Brera, Centro Storico, Conca del Naviglio, Gua..."
1,2,"Stazione Centrale, Gorla, Turro, Greco, Cresce...","Adriano, Crescenzago, Gorla, Greco, Loreto, Ma..."
2,3,"Città Studi, Lambrate, Porta Venezia","Casoretto, Cimiano, Città Studi, Dosso, Lambra..."
3,4,"Porta Vittoria, Forlanini","Acquabella, Calvairate, Castagnedo, Cavriano, ..."
4,5,"Vigentino, Chiaravalle, Gratosoglio","Basmetto, Cantalupa, Case Nuove, Chiaravalle, ..."


In [84]:
# Load local csv file containing latitude and longitude for each borough
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

@hidden_cell
#The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
#You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_8d0825c3ad5e45d992ecc8e3e8ca16fd = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_8d0825c3ad5e45d992ecc8e3e8ca16fd = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_8d0825c3ad5e45d992ecc8e3e8ca16fd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bOJZczT_TKkZ2KVEZv7QImXBDffEHKMP3t7OflCjwpwQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_8d0825c3ad5e45d992ecc8e3e8ca16fd)

body = client_8d0825c3ad5e45d992ecc8e3e8ca16fd.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-khbsoxd42hmacx',Key='Milan_GeospatialCoordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Borough,Latitude,Longitude
0,1,45.465362,9.188748
1,2,45.492814,9.203981
2,3,45.481547,9.218666
3,4,45.440969,9.217621
4,5,45.445495,9.183412


### 3. Methodolody
___


### 4. Results
___

### 5. Discussion and Conclusions
___